# Testing different algorythms on data set iris

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)

In [2]:
from sklearn.ensemble import IsolationForest

X, y = iris.data, iris.target
X.insert(loc=4, column='class', value=y)
X

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [3]:
setosa = X[X['class'] == 0].values
setosa

array([[5.1, 3.5, 1.4, 0.2, 0. ],
       [4.9, 3. , 1.4, 0.2, 0. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [4.6, 3.1, 1.5, 0.2, 0. ],
       [5. , 3.6, 1.4, 0.2, 0. ],
       [5.4, 3.9, 1.7, 0.4, 0. ],
       [4.6, 3.4, 1.4, 0.3, 0. ],
       [5. , 3.4, 1.5, 0.2, 0. ],
       [4.4, 2.9, 1.4, 0.2, 0. ],
       [4.9, 3.1, 1.5, 0.1, 0. ],
       [5.4, 3.7, 1.5, 0.2, 0. ],
       [4.8, 3.4, 1.6, 0.2, 0. ],
       [4.8, 3. , 1.4, 0.1, 0. ],
       [4.3, 3. , 1.1, 0.1, 0. ],
       [5.8, 4. , 1.2, 0.2, 0. ],
       [5.7, 4.4, 1.5, 0.4, 0. ],
       [5.4, 3.9, 1.3, 0.4, 0. ],
       [5.1, 3.5, 1.4, 0.3, 0. ],
       [5.7, 3.8, 1.7, 0.3, 0. ],
       [5.1, 3.8, 1.5, 0.3, 0. ],
       [5.4, 3.4, 1.7, 0.2, 0. ],
       [5.1, 3.7, 1.5, 0.4, 0. ],
       [4.6, 3.6, 1. , 0.2, 0. ],
       [5.1, 3.3, 1.7, 0.5, 0. ],
       [4.8, 3.4, 1.9, 0.2, 0. ],
       [5. , 3. , 1.6, 0.2, 0. ],
       [5. , 3.4, 1.6, 0.4, 0. ],
       [5.2, 3.5, 1.5, 0.2, 0. ],
       [5.2, 3.4, 1.4, 0.2, 0. ],
       [4.7, 3

In [6]:
versicolor, virginica = X[X['class'] == 1].values, X[X['class'] == 2].values

In [5]:
from sklearn.model_selection import train_test_split 

clf = IsolationForest(n_estimators=110, contamination=0.01)
X_train, X_test = train_test_split(setosa, test_size=0.2, random_state=10)

clf.fit(X_train)

preds_X = clf.predict(X_test)
sum([1 if pred==1 else 0 for pred in preds_X]) / preds_X.size

1.0

In [24]:
from sklearn.model_selection import KFold 

def nested_cv(X, Y, Z):
    n_estimators = [100, 150, 200]
    max_samples = [0.5, 0.07, 1.0]
    contaminations = [0.05, 0.1, 0.2]

    b_params = ()
    scores = []
    error_rates = []

    
    cv = KFold(n_splits=5, shuffle=True, random_state=0)

    for _, (train_idx, test_idx) in enumerate(cv.split(X)):
        X_train, X_test = X[train_idx], X[test_idx]
        inner_cv = KFold(n_splits=3, shuffle=True, random_state=0)

        b_f1 = 0

        for _, (train_idx1, valid_idx) in enumerate(inner_cv.split(X_train)):
            X_train1, X_valid = X_train[train_idx1], X_train[valid_idx]

            for n_est in n_estimators:
                for max_s in max_samples:
                    for contamination in contaminations:
                            
                        clf = IsolationForest(n_estimators=n_est, max_samples=max_s, contamination=contamination)
                        clf.fit(X_train1)

                        preds_X = clf.predict(X_valid)
                        preds_Y = clf.predict(Y)
                        preds_Z = clf.predict(Z)

                        tp = sum([1 if pred==1 else 0 for pred in preds_X])
                        tn = sum([1 if pred==-1 else 0 for pred in preds_Y]) + sum([1 if pred==-1 else 0 for pred in preds_Z])

                        f1_score = (2 * tp) / (preds_X.size + preds_Y.size + preds_Z.size + tp - tn)

                        if f1_score > b_f1:
                            b_f1 = f1_score
                            b_params = (n_est, max_s, contamination)


        clf = IsolationForest(n_estimators=b_params[0], max_samples=b_params[1], contamination=b_params[2])
        clf.fit(X_train)

        preds_X = clf.predict(X_test)
        preds_Y = clf.predict(Y)
        preds_Z = clf.predict(Z)

        score = sum([1 if pred==1 else 0 for pred in preds_X]) / preds_X.size 
        error_X = sum([1 if pred==-1 else 0 for pred in preds_X])
        error_Y = sum([1 if pred==1 else 0 for pred in preds_Y])
        error_Z = sum([1 if pred==1 else 0 for pred in preds_Z])

        error_rate = (error_X + error_Y + error_Z) / (preds_X.size + preds_Y.size + preds_Z.size)

        scores.append(score)
        error_rates.append(error_rate)
        
    return np.mean(scores), np.mean(error_rates), np.std(scores), np.std(error_rates), b_params

In [25]:
nested_cv(setosa, virginica, versicolor)

(0.9400000000000001,
 0.005454545454545454,
 0.07999999999999999,
 0.007272727272727272,
 (100, 0.5, 0.05))

In [26]:
nested_cv(virginica, setosa, versicolor)

(0.6799999999999999,
 0.05818181818181819,
 0.23151673805580456,
 0.016861124537264918,
 (200, 1.0, 0.2))

In [27]:
nested_cv(versicolor, virginica, setosa)

(0.76,
 0.04727272727272727,
 0.19595917942265423,
 0.008907235428302466,
 (100, 1.0, 0.1))

Le prestazioni sono molto simili all'algoritmo OneClassSVM, che generalmente è migliore: ha score maggiori nel caso di setosa e virgina ma score peggiore per versicolor, errori minori per setosa e verginica e invece errore minore per versicolor. Il tempo di esecuzione è, invece, nettamente inferiore quello dell'algoritmo OneClassSVM.
Da notare che non ho provato random_state differenti a causa del tempo di esecuzione troppo elevato, con l'algoritmo OneClassSVM, invece, avevo provato 20 random state differenti e poi avevo fatto la media.

In [32]:
from sklearn.neighbors import LocalOutlierFactor

def nested_cv_lof(X, Y, Z):
    n_neighbors = [5, 10, 15]
    contaminations = [0.05, 0.1, 0.2]

    b_params = ()
    scores = []
    error_rates = []

    for i in range(21):
        cv = KFold(n_splits=5, shuffle=True, random_state=i)

        for _, (train_idx, test_idx) in enumerate(cv.split(X)):
            X_train, X_test = X[train_idx], X[test_idx]

            b_f1 = 0
            inner_cv = KFold(n_splits=3, shuffle=True, random_state=i)

            for _, (train_idx1, valid_idx) in enumerate(inner_cv.split(X_train)):
                X_train1, X_valid = X_train[train_idx1], X_train[valid_idx]

                for neighs in n_neighbors:
                    for cont in contaminations:
                        clf = LocalOutlierFactor(n_neighbors=neighs, contamination=cont, novelty=True)
                        clf.fit(X_train1)

                        preds_X = clf.predict(X_valid)
                        preds_Y = clf.predict(Y)
                        preds_Z = clf.predict(Z)

                        tp = sum([1 if pred==1 else 0 for pred in preds_X])
                        tn = sum([1 if pred==-1 else 0 for pred in preds_Y]) + sum([1 if pred==-1 else 0 for pred in preds_Z])

                        f1_score = (2 * tp) / (preds_X.size + preds_Y.size + preds_Z.size + tp - tn)

                        if f1_score > b_f1:
                            b_f1 = f1_score
                            b_params = (neighs, cont)

            clf = LocalOutlierFactor(n_neighbors=b_params[0], contamination=b_params[1], novelty=True)
            clf.fit(X_train)

            preds_X = clf.predict(X_test)
            preds_Y = clf.predict(Y)
            preds_Z = clf.predict(Z)

            score = sum([1 if pred==1 else 0 for pred in preds_X]) / preds_X.size 
            error_X = sum([1 if pred==-1 else 0 for pred in preds_X])
            error_Y = sum([1 if pred==1 else 0 for pred in preds_Y])
            error_Z = sum([1 if pred==1 else 0 for pred in preds_Z])

            error_rate = (error_X + error_Y + error_Z) / (preds_X.size + preds_Y.size + preds_Z.size)

            scores.append(score)
            error_rates.append(error_rate)

    return np.mean(scores), np.mean(error_rates), np.std(scores), np.std(error_rates), b_params 
        

In [33]:
nested_cv_lof(setosa, versicolor, virginica)

(0.9409523809523811,
 0.005367965367965368,
 0.08129339936332684,
 0.007390309033029715,
 (5, 0.05))

In [34]:
nested_cv_lof(versicolor, setosa, virginica)

(0.9390476190476191,
 0.005541125541125541,
 0.09611029189746945,
 0.008737299263406311,
 (5, 0.05))

In [35]:
nested_cv_lof(virginica, setosa, versicolor)

(0.9190476190476192,
 0.007359307359307361,
 0.1015079054817084,
 0.009227991407428035,
 (5, 0.05))

Sul data set ottengo prestazioni migliori utilizzando l'algoritmo LocalOutlierFactor. Le prestazioni sono simili per quanto riguarda la classe setosa, sono però decisamente migliori per quando riguarda le restanti classi, versicolor e virginica.